##### Copyright 2019 Google LLC

- Adapted to own project, source: [github.com/GoogleCloudPlatform/cloudml-samples/](https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/scikit-learn/Training%20with%20scikit-learn%20in%20CMLE.ipynb)

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# scikit-learn Training on Google Cloud Machine Learning Engine
This notebook uses the [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) to demonstrate how to train a model on Cloud Machine Learning Engine (ML Engine).

# How to bring your model to ML Engine
Getting your model ready for training can be done in 3 steps:
1. Create your python model file
    1. Add code to download your data from [Google Cloud Storage](https://cloud.google.com/storage) so that ML Engine can use it
    1. Add code to export and save the model to [Google Cloud Storage](https://cloud.google.com/storage) once ML Engine finishes training the model
1. Prepare a package
1. Submit the training job

# Prerequisites
Before you jump in, let’s cover some of the different tools you’ll be using to get online prediction up and running on ML Engine. 

[Google Cloud Platform](https://cloud.google.com/) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[Cloud ML Engine](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.


# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable Cloud Machine Learning Engine and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [Install scikit-learn](http://scikit-learn.org/stable/install.html) [Optional: used if running locally]
* [Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html) [Optional: used if running locally]

These variables will be needed for the following steps.
* `TRAINER_PACKAGE_PATH <./census_training>` - A packaged training application that will be staged in a Google Cloud Storage location. The model file created below is placed inside this package path.
* `MAIN_TRAINER_MODULE <census_training.train>` - Tells ML Engine which file to execute. This is formatted as follows <folder_name.python_file_name>
* `JOB_DIR <gs://$BUCKET_NAME/scikit_learn_job_dir>` - The path to a Google Cloud Storage location to use for job output.
* `RUNTIME_VERSION <1.9>` - The version of Cloud ML Engine to use for the job. If you don't specify a runtime version, the training service uses the default Cloud ML Engine runtime version 1.0. [See the list of runtime versions for more information](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list).
* `PYTHON_VERSION <3.5>` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.

** Replace: **
* `PROJECT_ID <YOUR_PROJECT_ID>` - with your project's id. Use the PROJECT_ID that matches your Google Cloud Platform project.
* `BUCKET_NAME <YOUR_BUCKET_NAME>` - with the bucket id you created above.
* `JOB_DIR <gs://YOUR_BUCKET_NAME/scikit_learn_job_dir>` - with the bucket id you created above.
* `REGION <REGION>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'. The region is where the model will be deployed.

In [3]:
from utils import chdir_
pwd = chdir_()

import yaml
with open('config.yaml', 'r') as f:
    #cfg = yaml.load(f, Loader=yaml.BaseLoader)
    cfg = yaml.safe_load(f)

Current Working direcotory:	/home/enryh/gcp_poc/project


In [4]:
import os
pkg_dir = os.path.normpath('src/census_training')
%env PROJECT_ID {cfg['project-id']}
%env BUCKET_NAME {cfg['bucket']}
%env REGION {cfg['region']}
%env TRAINER_PACKAGE_PATH $pkg_dir
%env MAIN_TRAINER_MODULE census_training.train
%env JOB_DIR gs://{cfg['bucket']}/scikit_learn_job_dir
%env RUNTIME_VERSION 1.13
%env PYTHON_VERSION 3.5
!mkdir $pkg_dir

env: PROJECT_ID=ml-productive-pipeline-53122
env: BUCKET_NAME=ml-productive-pipeline-53122
env: REGION=europe-west1
env: TRAINER_PACKAGE_PATH=src/census_training
env: MAIN_TRAINER_MODULE=census_training.train
env: JOB_DIR=gs://ml-productive-pipeline-53122/scikit_learn_job_dir
env: RUNTIME_VERSION=1.13
env: PYTHON_VERSION=3.5
mkdir: cannot create directory ‘src/census_training’: File exists


##  The data
The [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) that this sample
uses for training is provided by the [UC Irvine Machine Learning
Repository](https://archive.ics.uci.edu/ml/datasets/). We have hosted the data on a public GCS bucket `gs://cloud-samples-data/ml-engine/sklearn/census_data/`. 

 * Training file is `adult.data`
 * Evaluation file is `adult.test` (not used in this notebook)

Note: Your typical development process with your own data would require you to upload your data to GCS so that ML Engine can access that data. However, in this case, we have put the data on GCS to avoid the steps of having you download the data from UC Irvine and then upload the data to GCS.

### Disclaimer
This dataset is provided by a third party. Google provides no representation,
warranty, or other guarantees about the validity or any other aspects of this dataset.

# Part 1: Create your python model file

First, we'll create the python model file (provided below) that we'll upload to ML Engine. This is similar to your normal process for creating a scikit-learn model. However, there are two key differences:
1. Downloading the data from GCS at the start of your file, so that ML Engine can access the data.
1. Exporting/saving the model to GCS at the end of your file, so that you can use it for predictions.

The code in this file loads the data into a pandas DataFrame that can be used by scikit-learn. Then the model is fit against the training data. Lastly, sklearn's built in version of joblib is used to save the model to a file that can be uploaded to [ML Engine's prediction service](https://cloud.google.com/ml-engine/docs/scikit/getting-predictions#deploy_models_and_versions).

**REPLACE Line 18: BUCKET_NAME = '<BUCKET_NAME>' with your GCS BUCKET_NAME**

Note: In normal practice you would want to test your model locally on a small dataset to ensure that it works, before using it with your larger dataset on ML Engine. This avoids wasted time and costs.

In [4]:
%%writefile $pkg_dir/train.py
# [START setup]
import datetime
import pandas as pd

from google.cloud import storage

from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer


# TODO: REPLACE '<BUCKET_NAME>' with your GCS BUCKET_NAME
BUCKET_NAME = 'ml-productive-pipeline-53122'
# [END setup]


# ---------------------------------------
# 1. Add code to download the data from GCS (in this case, using the publicly hosted data).
# ML Engine will then be able to use the data when training your model.
# ---------------------------------------
# [START download-data]
# Public bucket holding the census data
bucket = storage.Client().bucket('cloud-samples-data')

# Path to the data inside the public bucket
blob = bucket.blob('ml-engine/sklearn/census_data/adult.data')
# Download the data
blob.download_to_filename('adult.data')
# [END download-data]


# ---------------------------------------
# This is where your model code would go. Below is an example model using the census dataset.
# ---------------------------------------
# [START define-and-load-data]
# Define the format of your input data including unused columns (These are the columns from the census data files)
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)

# Categorical columns are columns that need to be turned into a numerical value to be used by scikit-learn
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)


# Load the training census dataset
with open('./adult.data', 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)

# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
train_features = raw_training_data.drop('income-level', axis=1).values.tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
train_labels = (raw_training_data['income-level'] == ' >50K').values.tolist()
# [END define-and-load-data]


# [START categorical-feature-conversion]
# Since the census data set has categorical features, we need to convert
# them to numerical values. We'll use a list of pipelines to convert each
# categorical column and then use FeatureUnion to combine them before calling
# the RandomForestClassifier.
categorical_pipelines = []

# Each categorical column needs to be extracted individually and converted to a numerical value.
# To do this, each categorical column will use a pipeline that extracts one feature column via
# SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
# A scores array (created below) will select and extract the feature column. The scores array is
# created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
for i, col in enumerate(COLUMNS[:-1]):
    if col in CATEGORICAL_COLUMNS:
        # Create a scores array to get the individual categorical column.
        # Example:
        #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 
        #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
        #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #
        # Returns: [['State-gov']]
        # Build the scores array
        scores = [0] * len(COLUMNS[:-1])
        # This column is the categorical column we want to extract.
        scores[i] = 1
        skb = SelectKBest(k=1)
        skb.scores_ = scores
        # Convert the categorical column to a numerical value
        lbn = LabelBinarizer()
        r = skb.transform(train_features)
        lbn.fit(r)
        # Create the pipeline to extract the categorical feature
        categorical_pipelines.append(
            ('categorical-{}'.format(i), Pipeline([
                ('SKB-{}'.format(i), skb),
                ('LBN-{}'.format(i), lbn)])))
# [END categorical-feature-conversion]

# [START create-pipeline]
# Create pipeline to extract the numerical features
skb = SelectKBest(k=6)
# From COLUMNS use the features that are numerical
skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
categorical_pipelines.append(('numerical', skb))

# Combine all the features using FeatureUnion
preprocess = FeatureUnion(categorical_pipelines)

# Create the classifier
classifier = RandomForestClassifier()

# Transform the features and fit them to the classifier
classifier.fit(preprocess.transform(train_features), train_labels)

# Create the overall model as a single pipeline
pipeline = Pipeline([
    ('union', preprocess),
    ('classifier', classifier)
])
# [END create-pipeline]


# ---------------------------------------
# 2. Export and save the model to GCS
# ---------------------------------------
# [START export-to-gcs]
# Export the model to a file
model = 'model.joblib'
joblib.dump(pipeline, model)

# Upload the model to GCS
bucket = storage.Client().bucket(BUCKET_NAME)
blob = bucket.blob('scikit_learn/{}/{}'.format(
    datetime.datetime.now().strftime('census_%Y%m%d_%H%M%S'),
    model))
blob.upload_from_filename(model)

Overwriting src/census_training/train.py


# Part 2: Create Trainer Package
Before you can run your trainer application with ML Engine, your code and any dependencies must be placed in a Google Cloud Storage location that your Google Cloud Platform project can access. You can find more info [here](https://cloud.google.com/ml-engine/docs/tensorflow/packaging-trainer)

In [5]:
%%writefile $pkg_dir/__init__.py
# Note that __init__.py can be an empty file.

Overwriting src/census_training/__init__.py


# Part 3: Submit Training Job
Next we need to submit the job for training on ML Engine. We'll use gcloud to submit the job which has the following flags:

* `job-name` - A name to use for the job (mixed-case letters, numbers, and underscores only, starting with a letter). In this case: `census_training_$(date +"%Y%m%d_%H%M%S")`
* `job-dir` - The path to a Google Cloud Storage location to use for job output.
* `package-path` - A packaged training application that is staged in a Google Cloud Storage location. If you are using the gcloud command-line tool, this step is largely automated.
* `module-name` - The name of the main module in your trainer package. The main module is the Python file you call to start the application. If you use the gcloud command to submit your job, specify the main module name in the --module-name argument. Refer to Python Packages to figure out the module name.
* `region` - The Google Cloud Compute region where you want your job to run. You should run your training job in the same region as the Cloud Storage bucket that stores your training data. Select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'.
* `runtime-version` - The version of Cloud ML Engine to use for the job. If you don't specify a runtime version, the training service uses the default Cloud ML Engine runtime version 1.0. See the list of runtime versions for more information.
* `python-version` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.
* `scale-tier` - A scale tier specifying the type of processing cluster to run your job on. This can be the CUSTOM scale tier, in which case you also explicitly specify the number and type of machines to use.

Note: Check to make sure gcloud is set to the current PROJECT_ID

In [9]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [14]:
import datetime
JOBNAME = 'census_training_'
JOBNAME += datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
%env JOBNAME $JOBNAME

env: JOBNAME=census_training_20190408_110644


In [15]:
!echo $JOBNAME

census_training_20190408_110644


Submit the training job.

In [16]:
! gcloud ml-engine jobs submit training $JOBNAME \
  --job-dir $JOB_DIR \
  --package-path $TRAINER_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version=$RUNTIME_VERSION \
  --python-version=$PYTHON_VERSION \
  --scale-tier BASIC

Job [census_training_20190408_110644] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe census_training_20190408_110644

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs census_training_20190408_110644
jobId: census_training_20190408_110644
state: QUEUED


In [17]:
!gcloud ml-engine jobs describe census_training_20190403_152433

createTime: '2019-04-03T13:24:36Z'
endTime: '2019-04-03T13:29:07Z'
etag: ESL4uGCPn3g=
jobId: census_training_20190403_152433
startTime: '2019-04-03T13:25:23Z'
state: SUCCEEDED
trainingInput:
  jobDir: gs://ml-productive-pipeline-53122/scikit_learn_job_dir
  packageUris:
  - gs://ml-productive-pipeline-53122/scikit_learn_job_dir/packages/8dde0283dc3d080f53da3cad00639cabf1c12f20d313c09121746b9f0a086782/census_training-0.0.0.tar.gz
  pythonModule: census_training.train
  pythonVersion: '3.5'
  region: europe-west1
  runtimeVersion: '1.13'
trainingOutput:
  consumedMLUnits: 0.07

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/census_training_20190403_152433?project=ml-productive-pipeline-53122

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2Fcensus_training_20190403_152433&project=ml-productive-pipeline-53122


# Verify Model File in GCS
View the contents of the destination model folder to verify that model file has indeed been uploaded to GCS.

Note: The model can take a few minutes to train and show up in GCS.

In [19]:
! gsutil ls gs://$BUCKET_NAME/scikit_learn/census_*

gs://ml-productive-pipeline-53122/scikit_learn/census_20190403_132638/model.joblib


# Use model with Predictions

## Test your model with local predictions

- local environment has to match selected [runtime](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)
- see [documentation](https://cloud.google.com/ml-engine/docs/scikit/using-pipelines-for-preprocessing#test_your_model_with_local_predictions)  

In [6]:
%env MODEL_DIR gs://{cfg['bucket']}/scikit_learn/{JOBNAME}/
%env MODEL_DIR gs://{cfg['bucket']}/scikit_learn/census_20190403_132638/ 
pwd = %pwd
INPUT_FILE=pwd + '/data/census/test_inputs.json'
%env INPUT_FILE $INPUT_FILE
FRAMEWORK="SCIKIT_LEARN"
%env FRAMEWORK $FRAMEWORK

env: MODEL_DIR=gs://{cfg['bucket']}/scikit_learn/{JOBNAME}/
env: MODEL_DIR=gs://ml-productive-pipeline-53122/scikit_learn/census_20190403_132638/
env: INPUT_FILE=/home/enryh/gcp_poc/project/data/census/test_inputs.json
env: FRAMEWORK=SCIKIT_LEARN


### Create some inputs

- ten first inputs from `data/census/adult.test` file

In [57]:
!head -11 data/census/adult.test

|1x3 Cross validator
25, Private, 226802, 11th, 7, Never-married, Machine-op-inspct, Own-child, Black, Male, 0, 0, 40, United-States, <=50K.
38, Private, 89814, HS-grad, 9, Married-civ-spouse, Farming-fishing, Husband, White, Male, 0, 0, 50, United-States, <=50K.
28, Local-gov, 336951, Assoc-acdm, 12, Married-civ-spouse, Protective-serv, Husband, White, Male, 0, 0, 40, United-States, >50K.
44, Private, 160323, Some-college, 10, Married-civ-spouse, Machine-op-inspct, Husband, Black, Male, 7688, 0, 40, United-States, >50K.
18, ?, 103497, Some-college, 10, Never-married, ?, Own-child, White, Female, 0, 0, 30, United-States, <=50K.
34, Private, 198693, 10th, 6, Never-married, Other-service, Not-in-family, White, Male, 0, 0, 30, United-States, <=50K.
29, ?, 227026, HS-grad, 9, Never-married, ?, Unmarried, Black, Male, 0, 0, 40, United-States, <=50K.
63, Self-emp-not-inc, 104626, Prof-school, 15, Married-civ-spouse, Prof-specialty, Husband, White, Male, 3103, 0, 32, United-States, >50K.
24, 

In [65]:
correct_label = [False, False, True, True, False, False, False, True, False, False]

In [7]:
%%writefile $INPUT_FILE
[25, "Private", 226802, "11th", 7, "Never-married", "Machine-op-inspct", "Own-child", "Black", "Male", 0, 0, 40, "United-States"]
[38, "Private", 89814, "HS-grad", 9, "Married-civ-spouse", "Farming-fishing", "Husband", "White", "Male", 0, 0, 50, "United-States"]
[28, "Local-gov", 336951, "Assoc-acdm", 12, "Married-civ-spouse", "Protective-serv", "Husband", "White", "Male", 0, 0, 40, "United-States"]
[44, "Private", 160323, "Some-college", 10, "Married-civ-spouse", "Machine-op-inspct", "Husband", "Black", "Male", 7688, 0, 40, "United-States"]
[18, "?", 103497, "Some-college", 10, "Never-married", "?", "Own-child", "White", "Female", 0, 0, 30, "United-States"]
[34, "Private", 198693, "10th", 6, "Never-married", "Other-service", "Not-in-family", "White", "Male", 0, 0, 30, "United-States"]
[29, "?", 227026, "HS-grad", 9, "Never-married", "?", "Unmarried", "Black", "Male", 0, 0, 40, "United-States"]
[63, "Self-emp-not-inc", 104626, "Prof-school", 15, "Married-civ-spouse", "Prof-specialty", "Husband", "White", "Male", 3103, 0, 32, "United-States"]
[24, "Private", 369667, "Some-college", 10, "Never-married", "Other-service", "Unmarried", "White", "Female", 0, 0, 40, "United-States"]
[55, "Private", 104996, "7th-8th", 4, "Married-civ-spouse", "Craft-repair", "Husband", "White", "Male", 0, 0, 10, "United-States"]

Overwriting /home/enryh/gcp_poc/project/data/census/test_inputs.json


### Sumit Inputs to model locally to machine which is running the notebook

In [62]:
!gcloud ml-engine local predict --model-dir $MODEL_DIR \
   --json-instances $INPUT_FILE \
   --framework $FRAMEWORK 


Operation completed over 1 objects/7.7 MiB.                                      

[False, False, False, True, False, False, False, False, False, False]


## Deploy Model
- using `gcloud ml-engine`
- set Environment Variables: `MODEL_NAME`, `VERSION_NAME`

In [13]:
%env VERSION_NAME v1
%env MODEL_NAME SKLEARN_CENSUS

# Create a Model resource
!gcloud ml-engine models create $MODEL_NAME --regions $REGION

Created ml engine model [projects/ml-productive-pipeline-53122/models/SKLEARN_CENSUS].


In [16]:
#Create a Model version
!gcloud ml-engine versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.13 \
  --framework $FRAMEWORK \
  --python-version=3.5

env: VERSION_NAME=v1
env: MODEL_NAME=SKLEARN_CENSUS
Creating version (this might take a few minutes)......done.                     


In [59]:
!gcloud ml-engine versions describe $VERSION_NAME \
  --model $MODEL_NAME

createTime: '2019-04-08T09:43:56Z'
deploymentUri: gs://ml-productive-pipeline-53122/scikit_learn/census_20190403_132638/
etag: -LrjegP6db8=
framework: SCIKIT_LEARN
isDefault: true
lastUseTime: '2019-04-08T11:05:31Z'
machineType: mls1-c1-m2
name: projects/ml-productive-pipeline-53122/models/SKLEARN_CENSUS/versions/v1
pythonVersion: '3.5'
runtimeVersion: '1.13'
state: READY


In [66]:
!gcloud ml-engine predict --model $MODEL_NAME --version \
  $VERSION_NAME --json-instances $INPUT_FILE

[False, False, False, True, False, False, False, False, False, False]


## Parse results and check correctness

In [67]:
result = !gcloud ml-engine predict --model $MODEL_NAME --version \
  $VERSION_NAME --json-instances $INPUT_FILE

In [102]:
print(result[0])
result=[x=="True" for x in result[0].strip("[]").split(", ")]

[False, False, False, True, False, False, False, False, False, False]


In [112]:

for predicted, correct in zip(result, correct_label):
    print("Predicted: {prediction}, Label: {label}".format(prediction=predicted, label=correct))

Predicted: False, Label: False
Predicted: False, Label: False
Predicted: False, Label: True
Predicted: True, Label: True
Predicted: False, Label: False
Predicted: False, Label: False
Predicted: False, Label: False
Predicted: False, Label: True
Predicted: False, Label: False
Predicted: False, Label: False


In [106]:
import sklearn.metrics as sklm
sklm.confusion_matrix(correct_label, result)

array([[7, 0],
       [2, 1]])

## Appendix

In [33]:
model_desc = !gcloud ml-engine versions describe $VERSION_NAME \
  --model $MODEL_NAME
model_desc[0]

"createTime: '2019-04-08T09:43:56Z'"

In [49]:
import json
model_desc_dict = {}
x = model_desc[0].split(": ")
for x in model_desc:
    x = x.split(": ")
    model_desc_dict[x[0]] = x[1]
model_desc_dict

{'createTime': "'2019-04-08T09:43:56Z'",
 'deploymentUri': 'gs://ml-productive-pipeline-53122/scikit_learn/census_20190403_132638/',
 'etag': '-LrjegP6db8=',
 'framework': 'SCIKIT_LEARN',
 'isDefault': 'true',
 'lastUseTime': "'2019-04-08T09:48:08Z'",
 'machineType': 'mls1-c1-m2',
 'name': 'projects/ml-productive-pipeline-53122/models/SKLEARN_CENSUS/versions/v1',
 'pythonVersion': "'3.5'",
 'runtimeVersion': "'1.13'",
 'state': 'READY'}